In [1]:
#lat/lon grid class
import sys
import os

from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch.client import indices

import ConfigParser

from sodapy import Socrata

#read in the config file
#os.chdir('~/Infrastructure_Capstone')
config = ConfigParser.ConfigParser()
config.read('../Infrastructure_Capstone/config/capstone_config.ini')

ES_url = config.get('ElasticSearch','host')
ES_password = config.get('ElasticSearch','password')
ES_username= config.get('ElasticSearch','username')

print ES_url

169.53.138.84


In [2]:
print sc

In [3]:
zip_codes = ['10065', '10069', '10453', '10452', '10451', '10457', '10456',
             '10455', '10454', '10459', '10458', '11232', '10103', '11233',
             '11385', '11234', '11235', '11222', '11237', '11218', '10282',
             '10280', '10119', '10115', '10112', '10110', '10111', '11109',
             '11379', '11378', '11375', '11374', '11377', '11101', '11371',
             '11370', '11373', '11372', '11102', '11223', '11103', '10803',
             '11229', '10040', '11106', '11228', '10044', '11201', '11203',
             '10128', '11205', '11104', '11207', '11226', '11209', '11208',
             '11105', '11225', '11224', '11368', '11369', '11366', '11367',
             '11364', '11365', '11362', '11363', '11360', '11361', '11221',
             '11220', '11040', '11212', '11213', '11210', '11211', '10039',
             '10038', '11214', '11215', '10035', '10034', '10037', '10036',
             '10031', '10030', '10033', '10032', '11359', '11358', '11005',
             '11004', '11697', '11351', '11357', '11356', '11355', '11354',
             '11216', '11435', '11451', '11217', '10309', '10308', '10307',
             '10306', '10305', '10304', '10303', '10302', '10301', '10028',
             '10029', '10026', '10027', '10024', '10025', '10022', '10023',
             '10020', '10021', '11429', '11219', '10271', '10168', '11230',
             '11428', '11421', '11420', '11423', '11422', '11425', '11424',
             '11427', '11426', '10152', '10153', '10154', '10310', '10311',
             '10312', '10314', '11001', '11231', '10009', '11238', '11239',
             '10019', '10018', '10013', '10012', '10011', '10010', '10017',
             '10016', '11236', '10014', '11693', '10002', '11432', '11433',
             '11430', '11436', '11694', '11434', '11204', '10004', '10005',
             '10006', '10007', '10001', '10169', '10003', '10167', '11692',
             '10165', '11206', '10162', '10279', '10278', '10475', '10474',
             '10471', '10470', '10473', '10472', '11691', '10075', '10170',
             '10171', '10172', '10173', '10174', '10177', '10466', '10467',
             '10464', '10465', '10462', '10463', '10460', '10461', '10468',
             '10469', '11411', '11412', '11413', '11414', '11415', '11416',
             '11417', '11418', '11419']

In [4]:
api_key = 'BWiAhuSnXE3WLESaDagpgOoxM'

def registration_counter(row):
    out = {}
    out['registered'] = 1
    out['scofflawed'] = 0 if row['scofflaw_indicator'] == 'N' else 1
    out['suspended'] = 0 if row['suspension_indicator'] == 'N' else 1
    out['revoked'] = 0 if row['revocation_indicator'] == 'N' else 1
    return out
    
    
def get_open_data(url,endpoint,limit,query=None):
    #input: Socrata endpoint for data, and OPTIONAL query
    client = Socrata(url,api_key)
    out = []
    idx=0
    data = client.get(endpoint,limit=limit,offset=idx,where=query)
    while len(data) > 0:
        #page through the results
        out.append(data)
        idx+=limit
        data = client.get(endpoint,limit=limit,offset=idx,where=query)

    return out
        
    
def get_car_registration_data():
    registration = sc.parallelize(get_open_data('data.ny.gov','w4pv-hbkt',limit=50000)) #load NY state auto registration data into an RDD
    
    #add up registered, scofflawed, suspended and revoked vehicles
    nyc_reg = (registration.filter(lambda row: row['zip'] in zip_codes and row['record_type'] == 'VEH')
                            .map(lambda row: (row['zip'],registration_counter(row)))
                            .reduceByKey(lambda x,y: {k:x[k] + y[k] for k in x})
              )
    
    return nyc_reg
    
    

In [ ]:
nyc_reg = get_car_registration_data()
nyc_reg.take(3)

In [ ]:
es_write_conf = {
        "es.nodes" : ES_url,
        "es.port" : "9200",
        "es.net.http.auth.user" : ES_username, 
        "es.net.http.auth.pass" : ES_password,
        "es.resource" : "dataframe_plus_weather/rows",
        "es.mapping.id" : "grid_id"
    } 

In [ ]:
updated_rdd.saveAsNewAPIHadoopFile(
            path='-', 
            outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
            keyClass="org.apache.hadoop.io.NullWritable", 
            valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
            conf=es_write_conf)